In [ ]:
import os
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torchvision import transforms
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score

# ========== CONFIG ========== #
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ROOT_TRAIN = "C:\\Users\\mrinmoy\\Documents\\GitHub\\hackthoncomsys_face_classification_challenge\\Comsys_Hackathon5\\Task_B\\train"  # UPDATE
ROOT_VAL = "C:\\Users\\mrinmoy\\Documents\\GitHub\\hackthoncomsys_face_classification_challenge\\Comsys_Hackathon5\\Task_B\\val"      # UPDATE
N_WAY = 5 ## 5 classes per episode
N_SHOT = 5 ## 5 distorted images per class (we have 7 available)
N_QUERY = 1 ## Number of query images per class as real images per class is 1
EPISODES = 2000
LR = 1e-3
IMG_SIZE = 224
EMBED_DIM = 256

# ========== TRANSFORM ========== #
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# ========== DATASET ========== #
class FewShotDataset:
    def __init__(self, root_dir, transform, n_shot=5, n_query=1):
        self.root_dir = root_dir
        self.transform = transform
        self.n_shot = n_shot
        self.n_query = n_query

        self.classes = []
        self.class_to_images = {}

        for cls in os.listdir(root_dir):
            class_path = os.path.join(root_dir, cls)
            if not os.path.isdir(class_path):
                continue

            clean_images = [os.path.join(class_path, img) for img in os.listdir(class_path)
                            if img.endswith('.jpg') and not os.path.isdir(os.path.join(class_path, img))]

            distortion_path = os.path.join(class_path, 'distortion')
            if os.path.exists(distortion_path):
                distorted_images = [os.path.join(distortion_path, img) for img in os.listdir(distortion_path)
                                    if img.endswith('.jpg')]
            else:
                distorted_images = []

            # Only keep classes that have enough images
            if len(clean_images) >= self.n_query and len(distorted_images) >= self.n_shot:
                self.classes.append(cls)
                self.class_to_images[cls] = {'clean': clean_images, 'distorted': distorted_images}

        if len(self.classes) == 0:
            raise Exception("No class has enough images. Please check dataset or adjust n_shot/n_query.")

    def sample_episode(self, n_way=5):
        """
        Sample a single N-way K-shot episode.
        Support: distorted images
        Query: clean images
        """
        if len(self.classes) < n_way:
            raise Exception(f"Not enough classes with sufficient images. Available: {len(self.classes)}, Requested: {n_way}")

        while True:
            selected_classes = random.sample(self.classes, n_way)

            support_images = []
            support_labels = []
            query_images = []
            query_labels = []

            label_mapping = {cls: idx for idx, cls in enumerate(selected_classes)}
            valid_episode = True

            for cls in selected_classes:
                clean_imgs = self.class_to_images[cls]['clean']
                distorted_imgs = self.class_to_images[cls]['distorted']

                if len(clean_imgs) < self.n_query or len(distorted_imgs) < self.n_shot:
                    valid_episode = False
                    break  # Resample episode

                support_samples = random.sample(distorted_imgs, self.n_shot)
                query_samples = random.sample(clean_imgs, self.n_query)

                for img_path in support_samples:
                    img = Image.open(img_path).convert('RGB')
                    support_images.append(self.transform(img))
                    support_labels.append(label_mapping[cls])

                for img_path in query_samples:
                    img = Image.open(img_path).convert('RGB')
                    query_images.append(self.transform(img))
                    query_labels.append(label_mapping[cls])

            if valid_episode and support_images and query_images:
                return (torch.stack(support_images), torch.tensor(support_labels),
                        torch.stack(query_images), torch.tensor(query_labels))

# ========== MODEL ========== #
class ProtoNet(nn.Module):
    def __init__(self, embed_dim=EMBED_DIM):
        super().__init__()
        base = models.resnet18(pretrained=True)
        self.features = nn.Sequential(*list(base.children())[:-1])
        self.fc = nn.Linear(base.fc.in_features, embed_dim)

    def forward(self, x):
        x = self.features(x).squeeze()
        return self.fc(x)

# ========== PROTOTYPICAL PREDICTION ========== #
def euclidean_dist(a, b):
    n = a.size(0)
    m = b.size(0)
    a = a.unsqueeze(1).expand(n, m, -1)
    b = b.unsqueeze(0).expand(n, m, -1)
    return torch.pow(a - b, 2).sum(2)

def predict_protonet(support_embeddings, support_labels, query_embeddings, n_way):
    prototypes = []
    for c in range(n_way):
        class_mask = (support_labels == c)
        class_embeddings = support_embeddings[class_mask]
        class_proto = class_embeddings.mean(0)
        prototypes.append(class_proto)
    prototypes = torch.stack(prototypes)
    dists = euclidean_dist(query_embeddings, prototypes)
    scores = -dists
    return scores

# ========== VALIDATION ========== #
def validate(model, val_dataset, device, n_way=5, n_shot=5, n_query=5, episodes=50):
    model.eval()
    accs = []

    with torch.no_grad():
        for _ in range(episodes):
            support_x, support_y, query_x, query_y = val_dataset.sample_episode(n_way)
            support_x, support_y = support_x.to(device), support_y.to(device)
            query_x, query_y = query_x.to(device), query_y.to(device)

            support_emb = model(support_x)
            query_emb = model(query_x)

            logits = predict_protonet(support_emb, support_y, query_emb, n_way)
            preds = logits.argmax(1)
            acc = (preds == query_y).float().mean().item()
            accs.append(acc)

    return sum(accs) / len(accs)

# ========== TRAINING ========== #
def train_protonet():
    train_dataset = FewShotDataset(ROOT_TRAIN, transform)
    val_dataset = FewShotDataset(ROOT_VAL, transform)

    model = ProtoNet().to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    best_val_acc = 0

    for episode in tqdm(range(EPISODES)):
        support_x, support_y, query_x, query_y = train_dataset.sample_episode(N_WAY)
        support_x, support_y = support_x.to(DEVICE), support_y.to(DEVICE)
        query_x, query_y = query_x.to(DEVICE), query_y.to(DEVICE)

        model.train()
        support_emb = model(support_x)
        query_emb = model(query_x)

        logits = predict_protonet(support_emb, support_y, query_emb, N_WAY)
        loss = F.cross_entropy(logits, query_y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (episode + 1) % 10 == 0:
            preds = logits.argmax(1)
            acc = (preds == query_y).float().mean().item()
            val_acc = validate(model, val_dataset, DEVICE, N_WAY, N_SHOT, N_QUERY, episodes=20)
            print(f"Episode {episode+1} | Loss: {loss.item():.4f} | Train Acc: {acc:.4f} | Val Acc: {val_acc:.4f}")

            if val_acc > best_val_acc:
                best_val_acc = val_acc
                torch.save(model.state_dict(), "best_protonet.pth")
                print(f"Best model saved at Episode {episode+1} with Val Acc: {val_acc:.4f}")

if __name__ == "__main__":
    train_protonet()


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 190MB/s]
  1%|          | 11/2000 [00:07<34:12,  1.03s/it]

Episode 10 | Loss: 0.0014 | Train Acc: 1.0000 | Val Acc: 0.4667
Best model saved at Episode 10 with Val Acc: 0.4667


  1%|          | 20/2000 [00:13<55:28,  1.68s/it]

Episode 20 | Loss: 1.3259 | Train Acc: 0.6667 | Val Acc: 0.7000
Best model saved at Episode 20 with Val Acc: 0.7000


  2%|▏         | 30/2000 [00:23<1:09:09,  2.11s/it]

...

Episode 1980 | Loss: 0.0000 | Train Acc: 1.0000 | Val Acc: 0.9000


100%|█████████▉| 1991/2000 [23:10<00:09,  1.02s/it]

Episode 1990 | Loss: 8.4743 | Train Acc: 0.3333 | Val Acc: 0.9500


100%|██████████| 2000/2000 [23:14<00:00,  1.43it/s]

Episode 2000 | Loss: 0.0002 | Train Acc: 1.0000 | Val Acc: 0.8833
